In [1]:
from typing import Annotated, Literal
import os 
from autogen import ConversableAgent, GroupChat, GroupChatManager

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
config_list = [{
    "model": "gpt_4o_0513",
    "base_url": "https://aoai-eaom-poc-gpt4o.openai.azure.com/",
    "api_type": "azure",
    "api_version": "2024-04-01-preview",
    "api_type": "azure",
    "api_key": os.getenv("AZURE_OPENAI_API_KEY")
    }]
llm_config = {"config_list": config_list, "temperature": 0,"cache_seed": 43, "timeout": 120,}
#llm_config = {"config_list": config_list, "timeout": 120,}

In [3]:
# The Number Agent always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [4]:
# The `description` attribute is a string that describes the agent.
# It can also be set in `ConversableAgent` constructor.
adder_agent.description = "Add 1 to each input number."
multiplier_agent.description = "Multiply each input number by 2."
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."
number_agent.description = "Return the numbers given."

In [8]:
group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=15, speaker_selection_method="auto"
)

In [9]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config, 
)

In [10]:
chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, I want to turn it into 13.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

14

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

5

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

10

--------------------------------------------------------------------------------

Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

6

-----------------------------------

In [5]:
group_chat_with_introductions = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=20,
    send_introductions=True,
)

In [6]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat_with_introductions,
    llm_config=llm_config, 
)

In [7]:
chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, I want to turn it into 13.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

5

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

-----------------------------------------------------------

# Constraint Transitions

In [8]:
allowed_transitions = {
    number_agent: [adder_agent, number_agent],
    adder_agent: [multiplier_agent, number_agent],
    subtracter_agent: [divider_agent, number_agent],
    multiplier_agent: [subtracter_agent, number_agent],
    divider_agent: [adder_agent, number_agent],
}

In [9]:
constrained_graph_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
    messages=[],
    max_round=12,
    send_introductions=True,
)

In [10]:
constrained_group_chat_manager = GroupChatManager(
    groupchat=constrained_graph_chat,
    llm_config=llm_config,
)

In [11]:
chat_result = number_agent.initiate_chat(
    constrained_group_chat_manager,
    message="My number is 3, I want to turn it into 10. Once I get to 10, keep it there.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 10. Once I get to 10, keep it there.

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

6

--------------------------------------------------------------------------------

Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

5

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

5

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager)

In [13]:
chat_result.chat_history

[{'content': 'My number is 3, I want to turn it into 10. Once I get to 10, keep it there.',
  'role': 'assistant'},
 {'content': 'Hello everyone. We have assembled a great team today to answer questions and solve tasks. In attendance are:\n\nAdder_Agent: Add 1 to each input number.\nMultiplier_Agent: Multiply each input number by 2.\nSubtracter_Agent: Subtract 1 from each input number.\nDivider_Agent: Divide each input number by 2.\nNumber_Agent: Return the numbers given.',
  'role': 'user'},
 {'content': '4', 'name': 'Adder_Agent', 'role': 'user'},
 {'content': '6', 'name': 'Multiplier_Agent', 'role': 'user'},
 {'content': '5', 'name': 'Subtracter_Agent', 'role': 'user'},
 {'content': '4', 'role': 'assistant'},
 {'content': '5', 'name': 'Adder_Agent', 'role': 'user'},
 {'content': '10', 'name': 'Multiplier_Agent', 'role': 'user'},
 {'content': '5', 'role': 'assistant'},
 {'content': '6', 'name': 'Adder_Agent', 'role': 'user'},
 {'content': '12', 'name': 'Multiplier_Agent', 'role': 'us